In [1]:
import stanza
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
import nltk
from nltk.corpus import stopwords
import numpy as np
import concurrent.futures

from thesis_utils import sentencer, deepest_sentence
nltk.download('stopwords')
stop = stopwords.words('spanish')

/Users/diegoacosta/Documents/Maestría/Thesis/Notebooks/Complete/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/dy/xr0vlym16y5cwngj_4p43nd40000gn/T/ipykernel_7938/2511470664.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/diegoacosta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Columns that we want to use
columns = ['MET', 'PRA', 'ASE', 'EVA', 'RET', 'APR', 'DOM', 'REC', 'MEJ', 'GÉNERO ALUMNO', 'PROM ACUMULADO EN PROFESIONAL', 'Género del profesor', 'Por qué no lo recomendarias', 'Comentarios']
aspects = ['MET', 'PRA', 'ASE', 'EVA', 'RET', 'APR', 'DOM', 'REC', 'MEJ']

In [3]:
# drive.mount('/content/drive')
# df=pd.read_excel('drive/MyDrive/Tesis/ecoa-demografico.xlsx')
df = pd.read_excel('./datasets/ecoa-demografico.xlsx')
# df = pd.read_csv('./datasets/df_toy.csv', index_col=0)

In [4]:
df.head()

,FOLIO,NOMINA,CRN course reference number,APR,ASE,DOM,EVA,MEJ,MET,PRA,...,% responsabilidad del profesor,Nacionalidad profesor,Campus,Materias que imparte el profesor,Semestres en TEC,Clase en inglés,Grado,TOEFL Profesor,Por qué no lo recomendarias,Comentarios
0,10022293,L00000049,7588,5.0,8.0,4.0,9.0,0.0,5.0,10.0,...,1.0,Mexicana,Monterrey,2.0,10.0,No,MaestrÌa,Mayor o igual 550,"Sabe explicar muy bien las cosas teoricas, per...",NaN
1,10221668,L00000049,7588,10.0,10.0,10.0,8.0,1.0,9.0,8.0,...,1.0,Mexicana,Monterrey,2.0,10.0,No,MaestrÌa,Mayor o igual 550,NaN,NaN
2,11705132,L00000049,7588,5.0,8.0,8.0,8.0,0.0,5.0,7.0,...,1.0,Mexicana,Monterrey,2.0,10.0,No,MaestrÌa,Mayor o igual 550,Método anticuado de enseñar. Los temas podría ...,NaN
3,11705156,L00000049,7588,10.0,10.0,10.0,10.0,0.0,10.0,10.0,...,1.0,Mexicana,Monterrey,2.0,10.0,No,MaestrÌa,Mayor o igual 550,NaN,NaN
4,11706514,L00000049,7588,10.0,10.0,10.0,10.0,0.0,8.0,10.0,...,1.0,Mexicana,Monterrey,2.0,10.0,No,MaestrÌa,Mayor o igual 550,NaN,NaN


In [5]:
print(f'Our original dataset is of shape: {df.shape}')

Our original dataset is of shape: (227746, 44)


In [6]:
df_ecoas = df[columns]
df_ecoas.head()

,MET,PRA,ASE,EVA,RET,APR,DOM,REC,MEJ,GÉNERO ALUMNO,PROM ACUMULADO EN PROFESIONAL,Género del profesor,Por qué no lo recomendarias,Comentarios
0,5.0,10.0,8.0,9.0,10.0,5.0,4.0,4.0,0.0,Femenino,94.428,Femenino,"Sabe explicar muy bien las cosas teoricas, per...",NaN
1,9.0,8.0,10.0,8.0,10.0,10.0,10.0,8.0,1.0,Femenino,91.280,Femenino,NaN,NaN
2,5.0,7.0,8.0,8.0,5.0,5.0,8.0,8.0,0.0,Femenino,95.968,Femenino,Método anticuado de enseñar. Los temas podría ...,NaN
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,Femenino,88.538,Femenino,NaN,NaN
4,8.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,0.0,Femenino,93.981,Femenino,NaN,NaN


In [7]:
pd.options.mode.chained_assignment = None # Disable warning
df_ecoas = df_ecoas.dropna(subset=['Por qué no lo recomendarias', 'Comentarios'], thresh=1) # Delete all rows where there are neither negative or positive comments
df_ecoas['GÉNERO ALUMNO'] = np.where((df_ecoas['GÉNERO ALUMNO'] == 'Femenino'), 0, df_ecoas['GÉNERO ALUMNO'])
df_ecoas['GÉNERO ALUMNO'] = np.where((df_ecoas['GÉNERO ALUMNO'] == 'Masculino'), 1, df_ecoas['GÉNERO ALUMNO'])
df_ecoas['Género del profesor'] = np.where((df_ecoas['Género del profesor'] == 'Femenino'), 0, df_ecoas['Género del profesor'])
df_ecoas['Género del profesor'] = np.where((df_ecoas['Género del profesor'] == 'Masculino'), 1, df_ecoas['Género del profesor'])

In [8]:
for asp in aspects: # We delete all incomplete submissions
  df_ecoas.loc[df_ecoas[asp].isnull(),asp] = 99
  df_ecoas = df_ecoas.loc[df_ecoas[asp] != 99]
df_ecoas = df_ecoas.reset_index(drop=True) # Reset the index

In [9]:
df_ecoas= df_ecoas.rename(columns={"Comentarios": "Positivos"})
df_ecoas= df_ecoas.rename(columns={"Por qué no lo recomendarias": "Negativos"})
p_only = df_ecoas['Positivos'].count()
n_only = df_ecoas['Negativos'].count()
print(f'Tenemos { p_only } comentarios SOLO positivos')
print(f'Tenemos { n_only } comentarios SOLO negativos')

Tenemos 87717 comentarios SOLO positivos
Tenemos 41954 comentarios SOLO negativos


In [10]:
df_ecoas['Tipo Comentario'] = 0
df_ecoas.loc[pd.notna(df_ecoas['Positivos']), 'Tipo Comentario'] = 1
df_ecoas.loc[pd.notna(df_ecoas['Negativos']) & pd.notna(df_ecoas['Positivos']), 'Tipo Comentario'] = 2


In [11]:
df_ecoas['Negativos'] = df_ecoas['Negativos'].fillna('')
df_ecoas['Positivos'] = df_ecoas['Positivos'].fillna('')
df_ecoas["Comentarios"] = df_ecoas["Positivos"] + ' ' + df_ecoas["Negativos"]
df_ecoas = df_ecoas.drop(columns=['Negativos', 'Positivos'])

In [12]:
df_ecoas['AVG'] = df_ecoas[['APR', 'EVA', 'MET', 'PRA', 'REC', 'RET']].mean(axis=1).round(1)

In [13]:
# This is the data set after initial cleaning
df_ecoas.head()

,MET,PRA,ASE,EVA,RET,APR,DOM,REC,MEJ,GÉNERO ALUMNO,PROM ACUMULADO EN PROFESIONAL,Género del profesor,Tipo Comentario,Comentarios,AVG
0,5.0,10.0,8.0,9.0,10.0,5.0,4.0,4.0,0.0,0,94.428,0,0,"Sabe explicar muy bien las cosas teoricas, pe...",7.2
1,5.0,7.0,8.0,8.0,5.0,5.0,8.0,8.0,0.0,0,95.968,0,0,Método anticuado de enseñar. Los temas podría...,6.3
2,8.0,10.0,8.0,5.0,10.0,10.0,10.0,10.0,1.0,0,96.408,0,1,"Sabe muchísimo del tema, muy preparada.",8.8
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,0,94.981,0,2,buena maestra si lo recomiendo,10.0
4,9.0,9.0,9.0,9.0,9.0,9.0,10.0,9.0,1.0,0,89.040,0,1,Tiene mucho conocimiento sobre los temas.,9.0


### Sentencer
*In this step we separate each comment in sentences using Spicy, by this, we will create a new column called -sentences- so we can later do sentiment analysis and topic classification over each sentence*

In [14]:
comentarios = df_ecoas['Comentarios']

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(sentencer, comentarios)

comments_sens = list(results)

In [16]:
comments_sens[0]

[[Sabe explicar muy bien las cosas teoricas, pero le hace falta aterrizar un poco los temas a lo que es en la practica la mercadotecnia interna.],
 [Nos puso a realizar un proyecto de mercadotecnia interna a una empresa sin nosotros tener algún ejemplo concreto de qué acciones se pueden implementar.],
 [Solo nos dio teoria basica.]]

In [17]:
df_ecoas['Sentences'] = comments_sens
df_ecoas.head()

,MET,PRA,ASE,EVA,RET,APR,DOM,REC,MEJ,GÉNERO ALUMNO,PROM ACUMULADO EN PROFESIONAL,Género del profesor,Tipo Comentario,Comentarios,AVG,Sentences
0,5.0,10.0,8.0,9.0,10.0,5.0,4.0,4.0,0.0,0,94.428,0,0,"Sabe explicar muy bien las cosas teoricas, pe...",7.2,"[[(Sabe, explicar, muy, bien, las, cosas, teor..."
1,5.0,7.0,8.0,8.0,5.0,5.0,8.0,8.0,0.0,0,95.968,0,0,Método anticuado de enseñar. Los temas podría...,6.3,"[[(Método, anticuado, de, enseñar, .)], [(Los,..."
2,8.0,10.0,8.0,5.0,10.0,10.0,10.0,10.0,1.0,0,96.408,0,1,"Sabe muchísimo del tema, muy preparada.",8.8,"[[(Sabe, muchísimo, del, tema, ,, muy, prepara..."
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,0,94.981,0,2,buena maestra si lo recomiendo,10.0,"[[(buena, maestra, si, lo, recomiendo)]]"
4,9.0,9.0,9.0,9.0,9.0,9.0,10.0,9.0,1.0,0,89.040,0,1,Tiene mucho conocimiento sobre los temas.,9.0,"[[(Tiene, mucho, conocimiento, sobre, los, tem..."


In [18]:
df_ecoas['Sentences'][0][0]

[Sabe explicar muy bien las cosas teoricas, pero le hace falta aterrizar un poco los temas a lo que es en la practica la mercadotecnia interna.]

### Lemmatization
*In this step we performed some NLP preprocessing techquines such as stop words removal, symbols and special characters removal and lemmatization of each of the columns and for each of the sentences*

In [19]:
comments = df_ecoas['Comentarios']

for a in range(len(comments)):
  if type(comments[a]) is not str:
    print(type(comments[a]), 'No es str: ', a)
    df_ecoas.drop(index=a)

comments = comments.str.lower()

nltk.download('stopwords')
stop = stopwords.words('spanish')
stop.remove('no')
stop.append('es')

# Tokenize the comments
for i in range(len(comments)):
  comments[i] = str(comments[i]).split()
  comments[i] = [word for word in comments[i] if word not in stop]

# Remove the symbols and replace them with blank spaces
sim = "!\"#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"

# Para cada símbolo, se remplaza por blank space
for c in range(len(comments)):
  for i in range(len(comments[c])):
    for j in sim:
      if comments[c][i].__contains__(j):
        comments[c][i] = comments[c][i].replace(j,'')

comments

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/diegoacosta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [sabe, explicar, bien, cosas, teoricas, hace, ...
1        [método, anticuado, enseñar, temas, podría, ha...
2                       [sabe, muchísimo, tema, preparada]
3                         [buena, maestra, si, recomiendo]
4                                    [conocimiento, temas]
                               ...                        
96535    [sabe, materia, gusta, enseñar, no, va, prisa,...
96536       [excelente, profesor, recomiendo, ampliamente]
96537    [profesor, llegó, salvar, semestre, pocas, cla...
96538    [dominio, tema, excelente, profesor, enseña, m...
96539    [triste, solo, pudo, dar, 3, clases, falto, ti...
Name: Comentarios, Length: 96540, dtype: object

In [23]:
total_less2  = 0
total_si = 0
total_no = 0

for comm in comments:
  if len(comm) <2:
    total_less2 += 1
    if 'si' in comm:
      total_si += 1
    if 'no' in comm:
      total_no += 1

print(f'Tenemos {total_less2} comentarios con una sola palabra, sin embargo, no los eliminaremos porque muchos son palabras que pueden ayudar al modelo')
print(f'Tenemos {total_si} comentarios con la palabra "si"')
print(f'Tenemos {total_no} comentarios con la palabra "no"')

Tenemos 2527 comentarios con una sola palabra, sin embargo, no los eliminaremos porque muchos son palabras que pueden ayudar al modelo
Tenemos 82 comentarios con la palabra "si"
Tenemos 30 comentarios con la palabra "no"


In [21]:
comments = list(comments)
for i in range(len(comments)):
  while '' in comments[i]:
    comments[i].remove('')

comentarios_merge=[None]*len(comments)

for i in range(len(comentarios_merge)):
  comentarios_merge[i]=(TreebankWordDetokenizer().detokenize(comments[i]))

print('Example of comment:')
comentarios_merge[2]

Example of comment:


'sabe muchísimo tema preparada'

In [24]:
comments_lemm = []

# Config the Stanza pipeline
config = {
	'processors': 'tokenize,mwt,pos,lemma',
  'lang': 'es',
}
nlp = stanza.Pipeline(**config) # Initialize the pipeline using a configuration dict

2024-03-21 21:41:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-03-21 21:41:52 INFO: Loading these models for language: es (Spanish):
| Processor | Package         |
-------------------------------
| tokenize  | ancora          |
| mwt       | ancora          |
| pos       | ancora_charlm   |
| lemma     | ancora_nocharlm |

2024-03-21 21:41:52 INFO: Using device: cpu
2024-03-21 21:41:52 INFO: Loading: tokenize
2024-03-21 21:41:52 INFO: Loading: mwt
2024-03-21 21:41:52 INFO: Loading: pos
2024-03-21 21:41:53 INFO: Loading: lemma
2024-03-21 21:41:53 INFO: Done loading processors!


In [25]:
com_lemmatized = []
comments_lemm = []

def lemmatize_comments(comment):
  doc = nlp(comment)
  lemmatized_comment = []

  for sent in doc.sentences:
    lemmatized_comment.extend([word.lemma for word in sent.words])
  return lemmatized_comment

with concurrent.futures.ThreadPoolExecutor() as executor:
  results = executor.map(lemmatize_comments, comentarios_merge)

comments_lemm = list(results)


In [26]:
print('Example of comment after lemmatizaion:')
comments_lemm[2]

Example of comment after lemmatizaion:


['saber', 'muchísimo', 'tema', 'preparado']

In [32]:
len(comments_lemm)

96540

In [33]:
df_ecoas['Lemm'] = comments_lemm
df_ecoas.head()

,MET,PRA,ASE,EVA,RET,APR,DOM,REC,MEJ,GÉNERO ALUMNO,PROM ACUMULADO EN PROFESIONAL,Género del profesor,Tipo Comentario,Comentarios,AVG,Sentences,Lemm
0,5.0,10.0,8.0,9.0,10.0,5.0,4.0,4.0,0.0,0,94.428,0,0,"Sabe explicar muy bien las cosas teoricas, pe...",7.2,"[[(Sabe, explicar, muy, bien, las, cosas, teor...","[saber, explicar, bien, cosa, teorico, hacer, ..."
1,5.0,7.0,8.0,8.0,5.0,5.0,8.0,8.0,0.0,0,95.968,0,0,Método anticuado de enseñar. Los temas podría...,6.3,"[[(Método, anticuado, de, enseñar, .)], [(Los,...","[método, anticuado, enseñar, tema, poder, hace..."
2,8.0,10.0,8.0,5.0,10.0,10.0,10.0,10.0,1.0,0,96.408,0,1,"Sabe muchísimo del tema, muy preparada.",8.8,"[[(Sabe, muchísimo, del, tema, ,, muy, prepara...","[saber, muchísimo, tema, preparado]"
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,0,94.981,0,2,buena maestra si lo recomiendo,10.0,"[[(buena, maestra, si, lo, recomiendo)]]","[buen, maestro, si, recomer]"
4,9.0,9.0,9.0,9.0,9.0,9.0,10.0,9.0,1.0,0,89.040,0,1,Tiene mucho conocimiento sobre los temas.,9.0,"[[(Tiene, mucho, conocimiento, sobre, los, tem...","[conocimiento, tema]"


In [34]:
df_ecoas.shape

(96540, 17)

### After lemmatization over the complete comments, we want to lemmatize and clean the comments but at the sentence-level

In [37]:
# stop.remove('no')
# stop.append('es')

def clean_sentence(sentence):
    sim = "!\"#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"
    sentence = sentence.lower()
    sentence = sentence.split()
    sentence = [word for word in sentence if word not in stop]
    
    for i in range(len(sentence)):
        for j in sim:
          if sentence[i].__contains__(j):
              sentence[i] = sentence[i].replace(j,'')
    while '' in sentence:
        sentence.remove('')
    return sentence

In [41]:
df_ecoas['Sentences'][0:10]

0    [[(Sabe, explicar, muy, bien, las, cosas, teor...
1    [[(Método, anticuado, de, enseñar, .)], [(Los,...
2    [[(Sabe, muchísimo, del, tema, ,, muy, prepara...
3             [[(buena, maestra, si, lo, recomiendo)]]
4    [[(Tiene, mucho, conocimiento, sobre, los, tem...
5    [[(Una, maestra, con, mucho, conocimiento, y, ...
6                           [[(Conocedora, de, tema)]]
7    [[(Aunque, no, la, conozco, fisicamente, ya, q...
8    [[(La, clase, esta, super, repetitiva, todo, e...
9    [[(Muchas, actividades, de, aprendizaje, .)], ...
Name: Sentences, dtype: object

In [46]:
# Recursively obtain the deepest sentence in the list of sentences

list_sentences = []

for i in range(len(df_ecoas['Sentences'])):
  sens = []
  for j in range(len(df_ecoas['Sentences'][i])):
    if type(df_ecoas['Sentences'][i][j]) is list:
        sentence_text = deepest_sentence(df_ecoas['Sentences'][i][j])
        sentence_text = clean_sentence(sentence_text) # Clean
        sentence_text = TreebankWordDetokenizer().detokenize(sentence_text) # merge the sentence back
        sentence_text = lemmatize_comments(sentence_text) # lemmatize
        sens.append(sentence_text)
  list_sentences.append(sens)

In [47]:
len(list_sentences)

96540

In [48]:
list_sentences[0]

[['saber',
  'explicar',
  'bien',
  'cosa',
  'teorico',
  'hacer',
  'falta',
  'aterrizar',
  'tema',
  'practicar',
  'mercadotecnia',
  'interno'],
 ['poner',
  'realizar',
  'proyecto',
  'mercadotecnia',
  'interno',
  'empresa',
  'tener',
  'alguno',
  'ejemplo',
  'concreto',
  'acción',
  'poder',
  'implementar'],
 ['solo', 'dar', 'teoria', 'basico']]

In [49]:
df_ecoas['Sentences_Lemm'] = list_sentences
df_ecoas.head()

,MET,PRA,ASE,EVA,RET,APR,DOM,REC,MEJ,GÉNERO ALUMNO,PROM ACUMULADO EN PROFESIONAL,Género del profesor,Tipo Comentario,Comentarios,AVG,Sentences,Lemm,Sentences_Lemm
0,5.0,10.0,8.0,9.0,10.0,5.0,4.0,4.0,0.0,0,94.428,0,0,"Sabe explicar muy bien las cosas teoricas, pe...",7.2,"[[(Sabe, explicar, muy, bien, las, cosas, teor...","[saber, explicar, bien, cosa, teorico, hacer, ...","[[saber, explicar, bien, cosa, teorico, hacer,..."
1,5.0,7.0,8.0,8.0,5.0,5.0,8.0,8.0,0.0,0,95.968,0,0,Método anticuado de enseñar. Los temas podría...,6.3,"[[(Método, anticuado, de, enseñar, .)], [(Los,...","[método, anticuado, enseñar, tema, poder, hace...","[[método, anticuado, enseñar], [tema, poder, h..."
2,8.0,10.0,8.0,5.0,10.0,10.0,10.0,10.0,1.0,0,96.408,0,1,"Sabe muchísimo del tema, muy preparada.",8.8,"[[(Sabe, muchísimo, del, tema, ,, muy, prepara...","[saber, muchísimo, tema, preparado]","[[saber, muchísimo, tema, preparado]]"
3,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,0,94.981,0,2,buena maestra si lo recomiendo,10.0,"[[(buena, maestra, si, lo, recomiendo)]]","[buen, maestro, si, recomer]","[[buen, maestro, si, recomer]]"
4,9.0,9.0,9.0,9.0,9.0,9.0,10.0,9.0,1.0,0,89.040,0,1,Tiene mucho conocimiento sobre los temas.,9.0,"[[(Tiene, mucho, conocimiento, sobre, los, tem...","[conocimiento, tema]","[[conocimiento, tema]]"


In [50]:
df_ecoas.to_csv('./datasets/df_ready.csv')